1. Установка необходимых библиотек:

In [1]:
pip install requests beautifulsoup4 pymongo pandas matplotlib 

Note: you may need to restart the kernel to use updated packages.


2. Импортирование библиотек: 

In [2]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt 

3. Получение данных через API: 

In [3]:
url = 'https://api.worldbank.org/v2/country/all/indicator/EN.ATM.CO2E.KT?format=json&date=2010:2020'
response = requests.get(url)
data = response.json()

url_gdp = 'https://api.worldbank.org/v2/country/all/indicator/NY.GDP.MKTP.CD?format=json&date=2010:2020'
response_gdp = requests.get(url_gdp)
data_gdp = response_gdp.json()

4. Извлечение данных и создание DataFrame: 

In [4]:
df_co2 = pd.json_normalize(data[1])
df_co2 = df_co2[['countryiso3code', 'date', 'value']]
df_co2.columns = ['country_code', 'year', 'co2_emissions']

df_gdp = pd.json_normalize(data_gdp[1])
df_gdp = df_gdp[['countryiso3code', 'date', 'value']]
df_gdp.columns = ['country_code', 'year', 'gdp']

5. Сохранение данных в SQLite:

In [5]:
conn = sqlite3.connect('world_bank_data.db')

df_co2.to_sql('co2_emissions', conn, if_exists='replace', index=False)
df_gdp.to_sql('gdp', conn, if_exists='replace', index=False)

50

7. Анализ данных с использованием SQLite: 

In [6]:
# зависимость между выбросами CO2 и ВВП
query = '''
SELECT co2.country_code, co2.year, co2.co2_emissions, gdp.gdp
FROM co2_emissions AS co2
JOIN gdp ON co2.country_code = gdp.country_code AND co2.year = gdp.year
'''
df_sqlite = pd.read_sql(query, conn)
print(df_sqlite) 

   country_code  year  co2_emissions           gdp
0           AFE  2020   5.449525e+05  9.290741e+11
1           AFE  2019   6.107235e+05  1.006527e+12
2           AFE  2018   5.987210e+05  1.012719e+12
3           AFE  2017   5.909055e+05  9.401055e+11
4           AFE  2016   5.802192e+05  8.298300e+11
5           AFE  2015   5.762660e+05  8.992957e+11
6           AFE  2014   5.916810e+05  9.796897e+11
7           AFE  2013   5.685475e+05  9.633473e+11
8           AFE  2012   5.467759e+05  9.526756e+11
9           AFE  2011   5.253379e+05  9.454391e+11
10          AFE  2010   5.326141e+05  8.494097e+11
11          AFW  2020   2.159156e+05  7.871467e+11
12          AFW  2019   2.229904e+05  8.239336e+11
13          AFW  2018   2.106189e+05  7.681896e+11
14          AFW  2017   2.005509e+05  6.857502e+11
15          AFW  2016   2.013992e+05  6.921811e+11
16          AFW  2015   1.943636e+05  7.693673e+11
17          AFW  2014   1.963436e+05  8.945854e+11
18          AFW  2013   1.86486

In [7]:
#страны с наибольшим увеличением выбросов CO2 за определённый период
query = '''
SELECT country_code, (MAX(co2_emissions) - MIN(co2_emissions)) AS co2_change
FROM co2_emissions
GROUP BY country_code
ORDER BY co2_change DESC
LIMIT 10;
'''
df_sqlite = pd.read_sql(query, conn)
print(df_sqlite) 

  country_code     co2_change
0          ARB  312169.770000
1          CEB   85593.280000
2          AFE   85385.610711
3          AFW   63416.393000
4          CSS    7700.590000


In [8]:
# ависимости между выбросами CO2 и ВВП для AFE
query = '''
SELECT co2.year, co2.co2_emissions, gdp.gdp
FROM co2_emissions AS co2
JOIN gdp ON co2.country_code = gdp.country_code AND co2.year = gdp.year
WHERE co2.country_code = 'AFE';
'''
df_sqlite = pd.read_sql(query, conn)
print(df_sqlite) 

    year  co2_emissions           gdp
0   2020  544952.503000  9.290741e+11
1   2019  610723.500000  1.006527e+12
2   2018  598720.957500  1.012719e+12
3   2017  590905.482000  9.401055e+11
4   2016  580219.242000  8.298300e+11
5   2015  576265.992000  8.992957e+11
6   2014  591680.972000  9.796897e+11
7   2013  568547.519000  9.633473e+11
8   2012  546775.865000  9.526756e+11
9   2011  525337.889289  9.454391e+11
10  2010  532614.140224  8.494097e+11
